In [ ]:
# !pip install praw
# !pip install requests-cache
# !pip install pymongo

In [2]:
import requests
import requests_cache
!mkdir -p web_cache
requests_cache.install_cache('web_cache/redditcrawler')
# requests_cache.clear()

In [3]:
import praw
from pymongo import MongoClient
import json
from datetime import datetime
from tqdm.notebook import tqdm
from IPython.display import JSON
from collections import Counter

from lib.parallel import parallel
from lib.labels_from_tags import labels_from_tags

In [4]:
mongo_uri = json.load(open('./credentials/mongodb_credentials.json'))['uri']
mongo = MongoClient(mongo_uri)
db = mongo['bad-vis']
reddit_raw = db['reddit-raw']
pushshift_raw = db['pushshift-raw']
reddit_merge = db['reddit-merge']
reddit_cache = db['reddit-cache']
posts = db['posts']

In [5]:
credentials = ""

with open('./credentials/reddit_credentials.json') as f:
    credentials = json.load(f)

In [6]:
reddit = praw.Reddit(client_id=credentials['client_id'],
                     client_secret=credentials['client_secret'],
                     password=credentials['password'],
                     user_agent=credentials['user_agent'],
                     username=credentials['username'])

In [7]:
# break the rate limiter since we have requests cache
# from types import MethodType

# def new_delay (self):
#     pass

# reddit._core._rate_limiter.delay = MethodType(new_delay, reddit._core._rate_limiter)

# Reddit

In [8]:
class PRAWJSONEncoder(json.JSONEncoder):
    """Class to encode PRAW objects to JSON."""
    """From https://gist.github.com/jarhill0/6e6495706252d52573950c3820f533b0"""
    def default(self, obj):
        if isinstance(obj, praw.models.base.PRAWBase):
            obj_dict = {}
            for key, value in obj.__dict__.items():
                if not key.startswith('_'):
                    obj_dict[key] = value
            return obj_dict
        else:
            return json.JSONEncoder.default(self, obj)

def digest_reddit_submission (submission):
    s = json.loads(json.dumps(submission, cls=PRAWJSONEncoder))
    if not reddit_raw.find_one({'id': submission.id}, {'_id': 1}):
        reddit_raw.insert_one(s)
    else:
        reddit_raw.update_one({'id': submission.id}, {'$set': s})
    return s

def reddit_submissions_iter (r, sort, limit):
    if sort == 'top_all':
        return reddit.subreddit(r).top('all', limit=limit)
    elif sort == 'top_year':
        return reddit.subreddit(r).top('year', limit=limit)
    else:
        return reddit.subreddit(r).new(limit=limit)

def retrieve_reddit_submissions (r='dataisugly', sort='top_all', limit=100):
    return (digest_reddit_submission(s) for s in tqdm(reddit_submissions_iter(r, sort, limit), total=limit))

In [9]:
# crawler_queue = [
#     {'r': 'dataisugly', 'sort': 'top_all', 'limit': 10}
# ]

reddit_crawler_queue = [
    {'r': 'dataisugly', 'sort': 'top_all', 'limit': 1000},
    {'r': 'dataisugly', 'sort': 'top_year', 'limit': 1000},
    {'r': 'dataisugly', 'sort': 'new', 'limit': 1000}
]

In [10]:
for c in reddit_crawler_queue:
    c['submissions'] = [s for s in retrieve_reddit_submissions(r=c['r'], sort=c['sort'], limit=c['limit'])]

In [11]:
datetime.fromtimestamp(reddit_crawler_queue[2]['submissions'][0]['created']).isoformat()

'2021-01-07T22:59:25'

# Pushshift

In [12]:
def digest_pushshift_submission (submission):
    if not pushshift_raw.find_one({'id': submission['id']}, {'_id': 1}):
        return pushshift_raw.insert_one(submission)
    else:
        return ''

def retrieve_pushshift (r, before_epoch, limit=1000):
    payload = {
        'subreddit': r,
        'sort': 'desc',
        'sort_type': 'created_utc',
        'before': before_epoch,
        'size': limit
    }
    reply = requests.get('https://api.pushshift.io/reddit/search/submission/', params=payload)
    if reply.status_code != 200:
        print(reply.status_code)
        print(json.dumps(payload))
    return reply.json()['data']

# def retrieve (r, before_epoch, limit=200):
#     api = PushshiftAPI(reddit)
#     submissions = api.search_submissions(before=before_epoch, subreddit=r, limit=limit)
#     return [s for s in submissions]

def pushshift_submissions_iter (r, limit=200):
    before_epoch = int(datetime(2021, 1, 1).timestamp())
    # before_epoch = int(datetime.now().timestamp()) # using now() will make the query payload keep shifting, bad for caching
    last_submission_id = 0
    while True:
        submissions = retrieve_pushshift(r, before_epoch, limit)
        if last_submission_id == submissions[-1]['id']:
            print(f"Last: {submissions[-1]['id']} {submissions[-1]['created_utc']} {datetime.fromtimestamp(submissions[-1]['created_utc']).isoformat()}")
            break

        for submission in submissions:
            yield submission

        before_epoch = submissions[-1]['created_utc'] + 1
        last_submission_id = submissions[-1]['id']
        print(f"New epoch: {before_epoch} {datetime.fromtimestamp(before_epoch).isoformat()}")

def retrieve_pushshift_submissions (r, limit=200):
    return (digest_pushshift_submission(s) for s in tqdm(pushshift_submissions_iter(r, limit)))

In [13]:
pushshift_crawler_queue = [
    {'r': 'dataisugly', 'limit': 1000}
]

In [14]:
for c in pushshift_crawler_queue:
    submissions = [s for s in retrieve_pushshift_submissions(r=c['r'], limit=c['limit'])]

New epoch: 1607098254 2020-12-04T16:10:54
New epoch: 1605098818 2020-11-11T12:46:58
New epoch: 1603206827 2020-10-20T15:13:47
New epoch: 1600739867 2020-09-22T01:57:47
New epoch: 1598532092 2020-08-27T12:41:32
New epoch: 1597342464 2020-08-13T18:14:24
New epoch: 1595523548 2020-07-23T16:59:08
New epoch: 1594004109 2020-07-06T02:55:09
New epoch: 1592565887 2020-06-19T11:24:47
New epoch: 1591403377 2020-06-06T00:29:37
New epoch: 1590017008 2020-05-20T23:23:28
New epoch: 1589025604 2020-05-09T12:00:04
New epoch: 1587664550 2020-04-23T17:55:50
New epoch: 1586451386 2020-04-09T16:56:26
New epoch: 1585659891 2020-03-31T13:04:51
New epoch: 1584536387 2020-03-18T12:59:47
New epoch: 1583287203 2020-03-04T02:00:03
New epoch: 1581922301 2020-02-17T06:51:41
New epoch: 1580127925 2020-01-27T12:25:25
New epoch: 1578134834 2020-01-04T10:47:14
New epoch: 1575557588 2019-12-05T14:53:08
New epoch: 1573142196 2019-11-07T15:56:36
New epoch: 1570254869 2019-10-05T05:54:29
New epoch: 1566666666 2019-08-24T1

# Merge to reddit-merge

In [27]:
reddit_merge.drop()
reddit_merge.create_index('id', unique=True)

'id_1'

In [28]:
def retrieve_submission (submission_id):
    s = reddit_cache.find_one({'id': submission_id})
    if s:
        return s

    try:
        s = reddit.submission(submission_id)
        name = s.name # trigger data fetching
        s = json.loads(json.dumps(s, cls=PRAWJSONEncoder))
    except Exception as e:
        print(submission_id, e)
        return None

    reddit_cache.update_one({'id': submission_id}, {'$setOnInsert': s}, upsert=True)
    return s

def add_merge (s, duplicated_with=None, retrieve=False):
    if not reddit_merge.find_one({'id': s['id']}, {'_id': 1}):
        if duplicated_with == None:
            duplicated_with = set()

        if 'crosspost_parent_list' in s:
            duplicated_with = duplicated_with.union([p['id'] for p in s['crosspost_parent_list']])

        if retrieve:
            submission = retrieve_submission(s['id'])
            if not submission:
                return 0
            if 'permalink' in s:
                submission['alt'] = s
            s = submission

        if 'crosspost_parent_list' in s:
            duplicated_with = duplicated_with.union([p['id'] for p in s['crosspost_parent_list']])
            del s['crosspost_parent_list']
        if 'alt' in s and 'crosspost_parent_list' in s['alt']:
            duplicated_with = duplicated_with.union([p['id'] for p in s['alt']['crosspost_parent_list']])
            del s['alt']['crosspost_parent_list']

        if 'reddit.com/r' in s['url']:
            tokens = s['url'].split('/')
            if len(tokens) > 6:
                duplicated_with.add(tokens[6])

        s['duplicated_with'] = list(duplicated_with)
        if '_id' in s:
            del s['_id']
        reddit_merge.update_one({'id': s['id']}, {'$setOnInsert': s}, upsert=True)

        added = [add_merge({'id': d}, duplicated_with=duplicated_with.union(set([s['id']])), retrieve=True) for d in duplicated_with]

        return sum(added) + 1

    return 0

In [29]:
result = parallel(add_merge, list(pushshift_raw.find()), params_dict={'retrieve': True}, total=pushshift_raw.estimated_document_count(), n_jobs=-1)
print(f"Retrieved {sum(result)}/{len(result)}")

bpca3e received 403 HTTP response
af71fc received 403 HTTP response

Retrieved 8826/7958


In [30]:
result = parallel(add_merge, reddit_raw.find(), total=reddit_raw.estimated_document_count(), n_jobs=-1)
print(f"Retrieved {sum(result)}/{len(result)}")


Retrieved 33/1721


In [31]:
# Check if duplicated posts inserted by racing condition
Counter([s['id'] for s in reddit_merge.find()]).most_common(5)

[('ehsr2o', 1), ('ehsqv3', 1), ('ehafnl', 1), ('eh9a95', 1), ('eh7yy8', 1)]

# Digest into posts

In [32]:
posts.delete_many({'source_platform': 'reddit'})

In [33]:
from collections.abc import MutableMapping, Sequence
from collections import Counter

# from https://stackoverflow.com/questions/51488240/python-get-json-keys-as-full-path
def get_paths(source):
    paths = []
    if isinstance(source, MutableMapping):  # found a dict-like structure...
        for k, v in source.items():  # iterate over it; Python 2.x: source.iteritems()
            paths.append([k])  # add the current child path
            paths += [[k] + x for x in get_paths(v)]  # get sub-paths, extend with the current
    # else, check if a list-like structure, remove if you don't want list paths included
    elif isinstance(source, Sequence) and not isinstance(source, str):
        #                          Python 2.x: use basestring instead of str ^
        for i, v in enumerate(source):
            paths.append([i])
            paths += [[i] + x for x in get_paths(v)]  # get sub-paths, extend with the current
    return paths

c = Counter([str(p) for s in reddit_merge.find() for p in get_paths(s)])
with open('reddit_attrs.txt', 'w') as f:
    f.write(str(c))

In [34]:
class Submission ():
    _attrs = [
        'id',
        'post_id',
        'datetime',
        'url',
        'title',
        'content',
        'author',
        'thumbnail',
        'preview',
        'removed',
        'ups',
        'num_comments',
        'external_link',
        'source',
        'source_platform',
        'source_url',
        'duplicated_posts',
        'tags',
        'media_type',
        'labels',
        'alt',
        'preview_alt',
        'thumbnail_alt',
        'external_link_alt'
    ]

    _video_type = {
        'gfycat.com',
        'streamable.com',
        'vimeo.com',
        'youtube.com'
    }

    def __init__ (self, s, duplicated_post=None):
        self._s = s
#         self._duplicated_post = duplicated_post
        self._subreddit = s['subreddit'] if type(s['subreddit']) == str else s['subreddit']['display_name']

        self.id = s['id']
        self.post_id = f'reddit/{self._subreddit}/{self.id}'
        self.title = s['title']
        self.content = s['selftext'] if 'selftext' in s else ''
        self.ups = s['ups'] if 'ups' in s else 0
        self.num_comments = s['num_comments']
        self.source = self._subreddit
        self.source_platform = 'reddit'
        self.source_url = f'https://www.reddit.com/{s["subreddit_name_prefixed"]}' if 'subreddit_name_prefixed' in s else ''

    def digest (self):
        return {a:getattr(self, a) for a in Submission._attrs}

    @property
    def alt (self):
        if 'alt' not in self._s:
            return None
        return Submission(self._s['alt']).digest()

    @property
    def preview_alt (self):
        if self.alt and self.alt['preview']['url'] and self.alt['preview']['url'] != self.preview['url']:
            return self.alt['preview']
        else:
            return None

    @property
    def thumbnail_alt (self):
        if self.alt and self.alt['thumbnail']['url'] and self.alt['thumbnail']['url'] != self.thumbnail['url']:
            return self.alt['thumbnail']
        else:
            return None

    @property
    def external_link_alt (self):
        if self.alt and self.alt['external_link'] and self.alt['external_link'] != self.external_link:
            return self.alt['external_link']
        else:
            return None

    @property
    def datetime (self):
        return datetime.fromtimestamp(self._s['created_utc']).isoformat()

    @property
    def url (self):
        if 'permalink' in self._s:
            if self._s['permalink'].startswith('/'):
                return f'https://reddit.com{self._s["permalink"]}'
            else:
                print(f'url: {self._s["permalink"]}')
                return self._s['permalink']
        else:
            return ''

    @property
    def author (self):
        if 'author' in self._s and self._s['author']:
            return self._s['author'] if type(self._s['author']) == str else self._s['author']['name']
        else:
            return ''

    @property
    def external_link (self):
        if 'url' in self._s and self._s['url'] != self.url:
            return self._s['url']
        else:
            return ''

    @property
    def duplicated_posts (self):
        duplicated_posts = []
        if 'duplicated_with' not in self._s:
            return duplicated_posts

        for submission_id in self._s['duplicated_with']:
            s = reddit_merge.find_one({'id': submission_id})
            if s:
                duplicated_posts.append(Submission(s).post_id)
        return duplicated_posts
#         duplicated_posts = []
#         if self._duplicated_post:
#             duplicated_posts.append(self._duplicated_post)
#         if self._crosspost:
#             duplicated_posts.append(self._crosspost.post_id)
#         if 'viz.wtf' in self.external_link:
#             wtfviz_id = [t for t in self.external_link.split('/') if t.isdigit()]
#             duplicated_posts.append(f'tumblr/wtf-viz/{wtfviz_id}')
#         return duplicated_posts

    @property
    def tags (self):
        return [self._s['link_flair_text']] if 'link_flair_text' in self._s and self._s['link_flair_text'] else []

    @property
    def labels (self):
        return {
            'auto': labels_from_tags(self.tags)
        }

    @property
    def media_type (self):
        if 'is_video' in self._s and self._s['is_video']:
            return 'video'
        if 'media' in self._s and self._s['media'] and 'type' in self._s['media']:
            if self._s['media']['type'] in Submission._video_type:
                return 'video'
        if self.preview['url']:
            return 'image'
        return 'text'

    @property
    def preview (self):
        try:
            if 'preview' in self._s:
                return {
                    'url': self._s['preview']['images'][0]['source']['url'],
                    'width': self._s['preview']['images'][0]['source']['width'],
                    'height': self._s['preview']['images'][0]['source']['height']
                }
            elif 'url' in self._s:
#                     print(f'Reddit submission preview digest no preview but url: {self._s["id"]} {self.url}')
                return {
                    'url': '',
                    'width': 0,
                    'height': 0
                }
            else:
#                     print(f'Reddit submission preview digest no preview nor url: {self._s["id"]}')
                return {
                    'url': '',
                    'width': 0,
                    'height': 0
                }
        except Exception as inst:
            print(f'Reddit submission preview digest error: {inst}')
            print(f'Reddit submission: {self._s["id"]} {self.url}')
            return {
                'url': '',
                'width': 0,
                'height': 0
            }

    @property
    def thumbnail (self):
        try:
            if 'thumbnail' in self._s:
                if not self._s['thumbnail'].startswith('http'):
                    if not (self._s['thumbnail'] == 'default' or
                            self._s['thumbnail'] == 'self' or
                            self._s['thumbnail'] == 'spoiler' or
                            self._s['thumbnail'] == 'nsfw' or
                            self._s['thumbnail'] == 'image'):
                        print(f"Reddit submission thumbnail invalid url: {self.id} {self._s['thumbnail']}")
                    return {
                        'url': '',
                        'width': 0,
                        'height': 0
                    }
                else:
                    return {
                        'url': self._s['thumbnail'],
                        'width': self._s.get('thumbnail_width', -1),
                        'height': self._s.get('thumbnail_height', -1)
                    }
            elif 'preview' in self._s:
                return {
                    'url': self._s['preview']['images'][0]['resolutions'][0]['url'],
                    'width': self._s['preview']['images'][0]['resolutions'][0]['width'],
                    'height': self._s['preview']['images'][0]['resolutions'][0]['height']
                }
            else:
                return {
                    'url': '',
                    'width': 0,
                    'height': 0
                }
        except Exception as inst:
            print(f'Reddit submission thumbnail digest error: {inst}')
            print(f'Reddit submission: {self._s["id"]} {self.url}')
            return {
                'url': '',
                'width': 0,
                'height': 0
            }

    @property
    def removed (self):
        return ('removed_by_category' in self._s and not not self._s['removed_by_category'])

In [35]:
def digest_all_submissions ():
    for s in tqdm(reddit_merge.find()):
        submission = Submission(s)
        posts.replace_one({'post_id': submission.post_id}, submission.digest(), upsert=True)
#         if submission._crosspost:
#             posts.replace_one({'post_id': submission._crosspost.post_id}, submission._crosspost.digest(), upsert=True)
    return

In [36]:
digest_all_submissions()

# Fix problematic posts

In [37]:
posts.find_one({'id': '4f927p'}) # invalid external_link: imgur.com/(null)

{'_id': ObjectId('5ff74a36095476d294c68320'),
 'id': '4f927p',
 'post_id': 'reddit/dataisugly/4f927p',
 'datetime': '2016-04-17T23:25:13',
 'url': 'https://reddit.com/r/dataisugly/comments/4f927p/the_truth_about_abuse/',
 'title': 'The Truth about Abuse',
 'content': '[deleted]',
 'author': '',
 'thumbnail': {'url': '', 'width': 0, 'height': 0},
 'preview': {'url': '', 'width': 0, 'height': 0},
 'removed': False,
 'ups': 0,
 'num_comments': 0,
 'external_link': 'http://imgur.com/(null)',
 'source': 'dataisugly',
 'source_platform': 'reddit',
 'source_url': 'https://www.reddit.com/r/dataisugly',
 'duplicated_posts': [],
 'tags': [],
 'media_type': 'text',
 'labels': {'auto': []},
 'alt': {'id': '4f927p',
  'post_id': 'reddit/dataisugly/4f927p',
  'datetime': '2016-04-17T23:25:13',
  'url': 'https://reddit.com/r/dataisugly/comments/4f927p/the_truth_about_abuse/',
  'title': 'The Truth about Abuse',
  'content': '',
  'author': 'sharkpony',
  'thumbnail': {'url': '', 'width': 0, 'height':

In [38]:
posts.update_one({'id': '4f927p'},
                 {'$set': {
                     'skip': True,
                     'crawler_remarks': 'invalid link: imgur.com/(null)'}})

In [39]:
posts.find_one({'external_link': 'http://[Imgur](http://i.imgur.com/ozwpju8.png)'}) # invalid external_link

{'_id': ObjectId('5ff74a39095476d294c68c34'),
 'id': '2zczew',
 'post_id': 'reddit/dataisugly/2zczew',
 'datetime': '2015-03-17T15:44:00',
 'url': 'https://reddit.com/r/dataisugly/comments/2zczew/the_cost_of_remediation_kind_of/',
 'title': 'The cost of remediation... kind of.',
 'content': '[deleted]',
 'author': '',
 'thumbnail': {'url': '', 'width': 0, 'height': 0},
 'preview': {'url': '', 'width': 0, 'height': 0},
 'removed': False,
 'ups': 1,
 'num_comments': 0,
 'external_link': 'http://[Imgur](http://i.imgur.com/ozwpju8.png)',
 'source': 'dataisugly',
 'source_platform': 'reddit',
 'source_url': 'https://www.reddit.com/r/dataisugly',
 'duplicated_posts': [],
 'tags': [],
 'media_type': 'text',
 'labels': {'auto': []},
 'alt': {'id': '2zczew',
  'post_id': 'reddit/dataisugly/2zczew',
  'datetime': '2015-03-17T15:44:00',
  'url': 'https://reddit.com/r/dataisugly/comments/2zczew/the_cost_of_remediation_kind_of/',
  'title': 'The cost of remediation... kind of.',
  'content': '',
  

In [40]:
posts.update_one({'external_link': 'http://[Imgur](http://i.imgur.com/ozwpju8.png)'},
                 {'$set': {
                     'external_link': 'http://i.imgur.com/ozwpju8.png',
                     'crawler_remarks': 'corrected invalid link'}})